<a href="https://colab.research.google.com/github/muhammadwajeeharif/DeepLearning-Assignment/blob/main/Credit_Card_Fraud_Detection_assignment_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Muhammad Wajeeh Arif
# PIAIC161905
# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras import models,layers
from keras import regularizers


In [ ]:
df = pd.read_csv('./creditcard.csv')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
# data["Class"].value_counts()
# x_data= data.drop(columns="Class", axis=0)
# label = data["Class"]

In [ ]:
# min_d = data.min()
# max_d = data.max()
# normalized_df=(data - min_d)/(max_d - min_d)
# normalized_df.head()

In [ ]:
train_data = data.sample(frac= 0.7, random_state=125)
test_data =  data.drop(train_data.index)

In [ ]:
train_labels = train_data.iloc[:,-1]
train_data = train_data.iloc[:,0:-1]
test_labels = test_data.iloc[:,-1]
test_data = test_data.iloc[:,0:-1]

In [ ]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [ ]:
train_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,...,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03
mean,-1.305197e-17,-1.148090e-17,4.451366e-16,1.381736e-16,-3.543368e-16,9.603671e-17,-3.714171e-17,1.817305e-16,-1.059465e-17,-5.024404e-16,...,-1.023210e-17,5.148277e-17,-4.161322e-17,4.785722e-17,-6.143288e-17,1.611354e-19,2.319846e-17,-3.879335e-17,1.643581e-17,-6.767688e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.877515e+00,-6.559463e+00,-7.975311e+00,-5.877864e+00,-1.939416e+00,-6.327695e+00,-3.713556e+00,-9.048185e+00,-1.092073e+01,-6.496406e+00,...,-6.894766e+00,-1.066741e+01,-8.850664e+00,-1.908527e+01,-3.254899e+00,-7.706496e+00,-3.400906e+00,-9.301447e+00,-1.288319e+01,-4.290544e-01
25%,-8.608714e-01,-8.403617e-02,-4.384486e-01,-6.102711e-02,-6.383382e-01,-9.932975e-02,-4.597244e-01,5.721196e-02,-8.546018e-02,-2.957751e-01,...,-3.236817e-01,-1.726429e-01,-5.685446e-01,-1.881379e-01,-6.337282e-01,-5.485544e-01,-6.551955e-01,-1.415273e-01,-1.932659e-01,-4.136222e-01
50%,-1.792874e-01,1.975596e-01,-1.628231e-01,2.988448e-01,-2.669480e-01,1.897611e-01,-4.158660e-02,2.668152e-01,-1.805987e-03,1.860241e-01,...,-1.394710e-01,-5.241965e-02,1.609477e-02,-5.732187e-03,1.074739e-01,4.602511e-02,-7.138666e-02,-2.934843e-02,7.814650e-03,-3.427025e-01
75%,9.472013e-01,5.604483e-01,1.608390e-01,5.195950e-01,3.897583e-01,4.207658e-01,3.948922e-01,4.227654e-01,1.252874e-01,5.675205e-01,...,1.602489e-01,9.758796e-02,5.404421e-01,1.916109e-01,7.351688e-01,5.970730e-01,5.915097e-01,2.515075e-01,3.224199e-01,-4.302323e-02
max,1.679250e+00,8.293915e-01,6.733141e+00,1.009087e+00,3.970632e+00,3.544377e+00,4.580162e+00,1.926572e+00,5.283599e+00,3.487510e+00,...,1.154049e+01,1.256578e+01,8.289939e+00,5.446041e+00,5.796762e+00,3.516774e+00,4.613447e+00,3.824109e+00,5.371044e+00,9.158452e+00


In [ ]:
#train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [ ]:
train_data = train_data.to_numpy()
train_labels = train_labels.to_numpy().astype('float64')

test_data = test_data.to_numpy()
test_labels = test_labels.to_numpy().astype('float64')

In [ ]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(train_data, train_labels, epochs=50,validation_split = 0.3)

Epoch 1/50
31/31 [==============================] - 2s 31ms/step - loss: 0.7633 - accuracy: 0.6651 - val_loss: 0.6913 - val_accuracy: 0.7295
Epoch 2/50
31/31 [==============================] - 0s 3ms/step - loss: 0.7382 - accuracy: 0.6977 - val_loss: 0.6670 - val_accuracy: 0.7295
Epoch 3/50
31/31 [==============================] - 0s 3ms/step - loss: 0.7668 - accuracy: 0.7070 - val_loss: 0.6444 - val_accuracy: 0.7295
Epoch 4/50
31/31 [==============================] - 0s 4ms/step - loss: 0.6775 - accuracy: 0.7445 - val_loss: 0.6218 - val_accuracy: 0.7295
Epoch 5/50
31/31 [==============================] - 0s 3ms/step - loss: 0.6695 - accuracy: 0.7129 - val_loss: 0.5994 - val_accuracy: 0.7295
Epoch 6/50
31/31 [==============================] - 0s 2ms/step - loss: 0.6655 - accuracy: 0.7354 - val_loss: 0.5832 - val_accuracy: 0.7295
Epoch 7/50
31/31 [==============================] - 0s 3ms/step - loss: 0.6230 - accuracy: 0.7453 - val_loss: 0.5721 - val_accuracy: 0.7295
Epoch 8/50
31/31 [=

In [ ]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 779us/step - loss: 0.3079 - accuracy: 0.9525


In [ ]:
loss

0.3079043924808502

In [ ]:
accuracy

0.9525423645973206

In [ ]:
predictions = model.predict(test_data)

In [ ]:
y_pred = (predictions > 0.5)

In [ ]:
con = tf.math.confusion_matrix(
    test_labels, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)

In [ ]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[446,   2],
       [ 26, 116]])>

In [ ]:
result = accuracy * 100
result

95.25423645973206